In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import io, math, os, sys
from base64 import b64decode
from pathlib import Path
from IPython.core.display import HTML

import matplotlib.pyplot as plt
import numpy as np

import PIL

# Install daltonlens if necessary
try:
    from daltonlens import convert, simulate, utils
except ImportError:
    !pip install -q daltonlens
    from daltonlens import convert, simulate, utils
    
# Uncomment to get interactive plots.
# %matplotlib notebook

# Introduction

Goal is to generate the precomputed matrices / parameters needed by libDaltonLens or DaltonLens desktop.

# Brettel 1997 with sRGB

In [19]:
print (utils.array_to_C_decl('LMS_from_linearRGB', convert.LMSModel_sRGB_SmithPokorny75().LMS_from_linearRGB))
print (utils.array_to_C_decl('linearRGB_from_LMS', convert.LMSModel_sRGB_SmithPokorny75().linearRGB_from_LMS))

static float LMS_from_linearRGB[] = {
    0.17882, 0.43516, 0.04119,
    0.03456, 0.27155, 0.03867,
    0.00030, 0.00184, 0.01467
};
static float linearRGB_from_LMS[] = {
    8.09444, -13.05043, 11.67206,
    -1.02485, 5.40193, -11.36147,
    -0.03653, -0.41216, 69.35132
};


In [18]:
simulator = simulate.Simulator_Brettel1997(convert.LMSModel_sRGB_SmithPokorny75(), use_white_as_neutral=True)
simulator.dumpPrecomputedValues = True
np.set_printoptions(precision=4, suppress=True)

simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.PROTAN, severity=1.0)
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.DEUTAN, severity=1.0)
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.TRITAN, severity=1.0)
print (utils.array_to_C_decl('brettel1997_tritan_normalSepPlaneLMS', simulator.n_sep_plane))
print (utils.array_to_C_decl('brettel1997_tritan_H1', simulator.H1))
print (utils.array_to_C_decl('brettel1997_tritan_H2', simulator.H2))


struct DLBrettel1997Params
{
    // Transformation using plane 1 == rgbFromLms . projection1 . lmsFromRgb
    float rgbCvdFromRgb_1[9];
    
    // Full transformation using plane 2 == rgbFromLms . projection2 . lmsFromRgb
    float rgbCvdFromRgb_2[9];

    // Normal of the separation plane to pick the right transform, already in the RGB space.
    // == normalInLms . lmsFromRgb
    float separationPlaneNormalInRgb[3];
};

static struct DLBrettel1997Params brettel_protan_params = {
    {
        0.14980, 1.19548, -0.34528,
        0.10764, 0.84864, 0.04372,
        0.00384, -0.00540, 1.00156,
    },
    {
        0.14570, 1.16172, -0.30742,
        0.10816, 0.85291, 0.03892,
        0.00386, -0.00524, 1.00139,
    },
    { 0.00048, 0.00393, -0.00441 }
};

struct DLBrettel1997Params
{
    // Transformation using plane 1 == rgbFromLms . projection1 . lmsFromRgb
    float rgbCvdFromRgb_1[9];
    
    // Full transformation using plane 2 == rgbFromLms . projection2 . lmsFromRgb
    float 

# Viénot 1999 with sRGB

In [15]:
simulator = simulate.Simulator_Vienot1999(convert.LMSModel_sRGB_SmithPokorny75())
simulator.dumpPrecomputedValues = True
np.set_printoptions(precision=4, suppress=True)

simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.PROTAN, severity=1.0)
print (utils.array_to_C_decl('vienot_projection_protan', simulator.lms_projection_matrix))

simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.DEUTAN, severity=1.0)
print (utils.array_to_C_decl('vienot_projection_deutan', simulator.lms_projection_matrix))

simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.TRITAN, severity=1.0)
print (utils.array_to_C_decl('vienot_projection_tritan', simulator.lms_projection_matrix))

static float vienot_protan_rgbCvd_from_rgb[] = {
    0.11238, 0.88762, 0.00000,
    0.11238, 0.88762, -0.00000,
    0.00401, -0.00401, 1.00000
};
static float vienot_projection_protan[] = {
    0.00000, 2.02344, -2.52580,
    0.00000, 1.00000, 0.00000,
    0.00000, 0.00000, 1.00000
};
static float vienot_deutan_rgbCvd_from_rgb[] = {
    0.29275, 0.70725, 0.00000,
    0.29275, 0.70725, -0.00000,
    -0.02234, 0.02234, 1.00000
};
static float vienot_projection_deutan[] = {
    1.00000, 0.00000, 0.00000,
    0.49421, 0.00000, 1.24827,
    0.00000, 0.00000, 1.00000
};
static float vienot_tritan_rgbCvd_from_rgb[] = {
    1.00000, 0.14461, -0.14461,
    0.00000, 0.85924, 0.14076,
    -0.00000, 0.85924, 0.14076
};
static float vienot_projection_tritan[] = {
    1.00000, 0.00000, 0.00000,
    0.00000, 1.00000, 0.00000,
    -0.01224, 0.07203, 0.00000
};


# Brettel with Vischeck parameters

In [5]:
simulator = simulate.Simulator_Vischeck()
simulator.dumpPrecomputedValues = True
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.PROTAN, severity=1.0)
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.DEUTAN, severity=1.0)
simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.TRITAN, severity=1.0)


struct DLBrettel1997Params
{
    // Transformation using plane 1 == rgbFromLms . projection1 . lmsFromRgb
    float rgbCvdFromRgb_1[9];
    
    // Full transformation using plane 2 == rgbFromLms . projection2 . lmsFromRgb
    float rgbCvdFromRgb_2[9];

    // Normal of the separation plane to pick the right transform, already in the RGB space.
    // == normalInLms . lmsFromRgb
    float separationPlaneNormalInRgb[3];
};

static struct DLBrettel1997Params brettel_protan_params = {
    {
        0.20611, 0.99192, -0.19803,
        0.16690, 0.79147, 0.04163,
        0.00967, -0.01209, 1.00241,
    },
    {
        0.20993, 1.01140, -0.22133,
        0.16609, 0.78738, 0.04653,
        0.00963, -0.01232, 1.00270,
    },
    { 0.00124, 0.00632, -0.00756 }
};

struct DLBrettel1997Params
{
    // Transformation using plane 1 == rgbFromLms . projection1 . lmsFromRgb
    float rgbCvdFromRgb_1[9];
    
    // Full transformation using plane 2 == rgbFromLms . projection2 . lmsFromRgb
    float 

array([[[0, 0, 0]]], dtype=uint8)